In [1]:
# Import all dependencies 

import os
import csv

import requests
import json
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sqlalchemy import create_engine

In [2]:
# Read in CSV file for home prices and view the file 

csv_file = "/Users/lauraehallaman/Documents/GitHub/housingandavocados/ASPUS.csv"
house_price_df = pd.read_csv(csv_file)
house_price_df.head()

,DATE,ASPUS
0,Q1 2010,275300
1,Q2 2010,268800
2,Q3 2010,266000
3,Q4 2010,278000
4,Q1 2011,268100


In [3]:
# Rename Columns 

house_q = house_price_df.rename(columns = {"DATE": "Quarter"})
house_q.head()

,Quarter,ASPUS
0,Q1 2010,275300
1,Q2 2010,268800
2,Q3 2010,266000
3,Q4 2010,278000
4,Q1 2011,268100


In [4]:
# Read in CSV file for avocado prices and view the file 

avocado_csv = "/Users/lauraehallaman/Documents/GitHub/housingandavocados/avocado.csv"

avocado_pd = pd.read_csv(avocado_csv)
avocado_pd.head()

,Unnamed: 0,Date,Quarter,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,51.0,1/4/15,Q1 2015,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015.0,Albany
1,51.0,1/4/15,Q1 2015,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015.0,Atlanta
2,51.0,1/4/15,Q1 2015,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015.0,BaltimoreWashington
3,51.0,1/4/15,Q1 2015,1.01,80034.32,44562.12,24964.23,2752.35,7755.62,6064.30,1691.32,0.0,conventional,2015.0,Boise
4,51.0,1/4/15,Q1 2015,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015.0,Boston


In [5]:
# Drop unneeded columns from the Avocado pandas dataframe
simplified_avocado = avocado_pd.loc[:,["Quarter", "AveragePrice", "Total Volume", "type", "region"]]
simplified_avocado.dropna()


,Quarter,AveragePrice,Total Volume,type,region
0,Q1 2015,1.22,40873.28,conventional,Albany
1,Q1 2015,1.00,435021.49,conventional,Atlanta
2,Q1 2015,1.08,788025.06,conventional,BaltimoreWashington
3,Q1 2015,1.01,80034.32,conventional,Boise
4,Q1 2015,1.02,491738.00,conventional,Boston
...,...,...,...,...,...
18244,Q1 2018,1.04,14503.47,organic,Syracuse
18245,Q1 2018,1.41,10028.49,organic,Tampa
18246,Q1 2018,1.55,1559967.20,organic,TotalUS
18247,Q1 2018,1.60,271723.08,organic,West


In [6]:
# Further clean the data by removing regional data not needed for our table 

us_avocado = simplified_avocado[simplified_avocado["region"] == "TotalUS"]

us_avocado.head()

,Quarter,AveragePrice,Total Volume,type,region
51,Q1 2015,0.95,31324277.73,conventional,TotalUS
105,Q1 2015,1.46,612910.15,organic,TotalUS
159,Q1 2015,1.01,29063542.75,conventional,TotalUS
213,Q1 2015,1.42,669528.88,organic,TotalUS
267,Q1 2015,1.03,29043458.85,conventional,TotalUS


In [7]:
# Limit results to conventional avocado sales 

conventional_avocado = us_avocado[us_avocado["type"] == "conventional"]
conventional_avocado.head()

,Quarter,AveragePrice,Total Volume,type,region
51,Q1 2015,0.95,31324277.73,conventional,TotalUS
159,Q1 2015,1.01,29063542.75,conventional,TotalUS
267,Q1 2015,1.03,29043458.85,conventional,TotalUS
375,Q1 2015,1.04,28470310.84,conventional,TotalUS
483,Q1 2015,0.89,44655461.51,conventional,TotalUS


In [8]:
#Calculate the average price and display the total volume by quarter; reset the index 

mean_avocado = conventional_avocado.groupby(["Quarter"])
mean_avocado_sum = pd.DataFrame(mean_avocado.mean().reset_index())

mean_avocado_sum.head()

,Quarter,AveragePrice,Total Volume
0,Q1 2015,1.000000,3.136929e+07
1,Q1 2016,0.911538,3.672442e+07
2,Q1 2017,1.016923,3.843227e+07
3,Q1 2018,1.060000,4.212553e+07
4,Q2 2015,1.026923,3.446435e+07


In [9]:
# Merge the two dataframes, average house prices and avocado price and sales, and then display

avocado_houses = pd.merge(mean_avocado_sum, house_q, on = "Quarter", how='outer')
avocado_houses = avocado_houses.rename(columns = {"Quarter": "quarter",
                                       "AveragePrice": "avg_avocado_price",
                                       "Total Volume": "avocado_sales_volume",
                                       "ASPUS": "avg_home_price"})

avocado_houses = avocado_houses.round({"avg_avocado_price": 2})
avocado_houses.head()

,quarter,avg_avocado_price,avocado_sales_volume,avg_home_price
0,Q1 2015,1.00,3.136929e+07,348000
1,Q1 2016,0.91,3.672442e+07,357000
2,Q1 2017,1.02,3.843227e+07,374800
3,Q1 2018,1.06,4.212553e+07,374600
4,Q2 2015,1.03,3.446435e+07,339700


In [10]:
# Drop the NaN values from the dataframe 

avocado_houses = avocado_houses.dropna()
avocado_houses

,quarter,avg_avocado_price,avocado_sales_volume,avg_home_price
0,Q1 2015,1.00,3.136929e+07,348000
1,Q1 2016,0.91,3.672442e+07,357000
2,Q1 2017,1.02,3.843227e+07,374800
3,Q1 2018,1.06,4.212553e+07,374600
4,Q2 2015,1.03,3.446435e+07,339700
5,Q2 2016,0.93,3.879472e+07,357900
6,Q2 2017,1.20,3.721404e+07,376900
7,Q3 2015,1.05,3.151440e+07,347400
8,Q3 2016,1.13,3.388143e+07,358800
9,Q3 2017,1.40,3.088223e+07,373200


In [11]:
# Create the engine and connection to PostgreSQL

rds_connection_string = "postgres:Booboo88@localhost:5432/avocado_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
# Display the table name 
engine.table_names()

['avocadohouses']

In [14]:
# Transmit the data from the pandas dataframe to the SQL database we created 

avocado_houses.to_sql(name='avocadohouses', con=engine, if_exists='append', index=False)